In [2]:

import os
import pandas as pd
import pandera as pa
from pandera import Column, Check, DataFrameSchema

#  File paths
base_dir = r"C:\Users\chang\Desktop\MS2 Platform Technology"
raw_path = os.path.join(base_dir, "marketing_summary.csv")
clean_path = os.path.join(base_dir, "CLEANED_marketing_summary.csv")

# 1 Load raw data & normalize column names
df = pd.read_csv(raw_path)
print("Columns before normalization:", df.columns.tolist())
# Standardize names
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(r"[ \-]+", "_", regex=True)
)
print("Columns after normalization:", df.columns.tolist())

# 2 Detect key columns
def find_column(candidates, cols):
    for c in candidates:
        if c in cols:
            return c
    return None

id_col   = find_column(["campaign_id", "id", "campaign"], df.columns)
date_col = find_column(["campaign_date", "date", "report_date"], df.columns)
if id_col is None and date_col is not None:
    print("⚠️ No explicit ID column found; using date as identifier.")
    id_col = date_col
if date_col is None:
    raise KeyError(f"Date column missing. Available: {df.columns.tolist()}")
print(f"Using id_col='{id_col}', date_col='{date_col}'")

# 3 Initial shape and missing-value overview
print(f"▶️ Raw data shape: {df.shape}")
print("Missing % per column:")
print((df.isnull().mean() * 100).sort_values(ascending=False))

# 4 Drop high-null & irrelevant columns
high_null = df.columns[df.isnull().mean() > 0.5]
pattern_cols = [c for c in df.columns if c.startswith("col_")]
# Exclude essential
drop_cols = [c for c in list(high_null) + pattern_cols if c not in [id_col, date_col]]
df = df.drop(columns=drop_cols, errors="ignore")
print(f"After dropping cols, shape: {df.shape}")

# Recompute numeric and categorical columns after drop
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = [c for c in df.select_dtypes(include=["object"]).columns if c not in [id_col, date_col]]
print(f"Numeric columns to clean: {numeric_cols}")
print(f"Categorical columns to clean: {cat_cols}")

# 5 Parse & clean key fields
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df = df.dropna(subset=[date_col])  # ensure date exists
df = df.drop_duplicates()
print(f"After date parsing & duplicate clean, shape: {df.shape}")

# 6 Numeric & categorical fixes
# Numeric: cast to float and remove negatives
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype(float)
    df = df[df[col].ge(0)]
# Categorical: strip, lower, cast to category
for col in cat_cols:
    df[col] = df[col].astype(str).str.strip().str.lower().astype("category")
print(f"After numeric & category clean, shape: {df.shape}")

# 7 Outlier filtering on numeric metrics (1.5×IQR)
for col in numeric_cols:
    q1, q3 = df[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    df = df[df[col].between(lower, upper)]
print(f"After outlier filtering, shape: {df.shape}")

# 8 Define & apply Pandera schema
schema_cols = {
    id_col: Column(str, nullable=False),
    date_col: Column(pa.DateTime, nullable=False)
}
for col in numeric_cols:
    schema_cols[col] = Column(float, Check.ge(0), nullable=True)
for col in cat_cols:
    schema_cols[col] = Column(pa.Category, nullable=True)
schema = DataFrameSchema(schema_cols)
df = schema.validate(df, lazy=True)
print("✔️ Pandera validation passed")

# 9 Save cleaned data
df.to_csv(clean_path, index=False)
print(f"✅ Cleaned data saved to: {clean_path}")


Columns before normalization: ['date', 'users_active', 'total_sales', 'new_customers', 'report_generated', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14', 'col_15', 'col_16', 'col_17', 'col_18', 'col_19', 'col_20', 'col_21', 'col_22', 'col_23', 'col_24', 'col_25', 'col_26', 'col_27', 'col_28', 'col_29', 'col_30', 'col_31', 'col_32', 'col_33', 'col_34', 'col_35', 'col_36', 'col_37', 'col_38', 'col_39', 'col_40', 'col_41', 'col_42', 'col_43', 'col_44', 'col_45', 'col_46', 'col_47', 'col_48', 'col_49', 'col_50']
Columns after normalization: ['date', 'users_active', 'total_sales', 'new_customers', 'report_generated', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14', 'col_15', 'col_16', 'col_17', 'col_18', 'col_19', 'col_20', 'col_21', 'col_22', 'col_23', 'col_24', 'col_25', 'col_26', 'col_27', 'col_28', 'col_29', 'col_30', 'col_31', 'col_32', 'col_33', 'col_34', 'col_35', 'col_36', 'col_37', 'col_38', 'col_39', 'col_

In [3]:
import os
import logging
import pandas as pd
import pandera as pa
from pandera import Column, Check, DataFrameSchema

# Setup minimal logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger(__name__)

# File paths
base_dir = r"C:\Users\chang\Desktop\MS2 Platform Technology"
raw_path = os.path.join(base_dir, "marketing_summary.csv")

# 1) Load data with error handling
try:
    df = pd.read_csv(raw_path)
    logger.info(f"Loaded marketing_summary with shape {df.shape}")
    error_handling = "Passed"
except Exception as e:
    logger.error(f"Failed to load marketing_summary: {e}")
    df = pd.DataFrame()
    error_handling = f"Failed ({e})"

# Prepare health‐check report
report = {
    "Missing/Broken columns detected": None,
    "Schema double check":             None,
    "Error Handling check":            error_handling
}

# 2) Detect missing or broken columns
required = ["campaign_id", "campaign_date"]
missing  = [col for col in required if col not in df.columns]
if missing:
    report["Missing/Broken columns detected"] = missing
    logger.warning(f"Required columns missing: {missing}")
else:
    report["Missing/Broken columns detected"] = []
    logger.info("All required columns present")

# 3) Define Pandera schema for double-check
schema = DataFrameSchema({
    "campaign_id":   Column(str, nullable=False),
    "campaign_date": Column(pa.DateTime, nullable=False),
    **{
        col: Column(float, Check.ge(0), nullable=True)
        for col in df.select_dtypes(include="number").columns
    },
})

# 4) Run schema validation, capture outcome
if not df.empty and not missing:
    df["campaign_date"] = pd.to_datetime(df["campaign_date"], errors="coerce")
    try:
        schema.validate(df, lazy=True)
        report["Schema double check"] = "Passed"
        logger.info("Schema validation passed")
    except pa.errors.SchemaErrors as err:
        report["Schema double check"] = f"Failed: {len(err.failure_cases)} errors"
        logger.error("Schema validation failed:\n%s", err.failure_cases)
else:
    report["Schema double check"] = "Skipped"

# 5) Print consolidated health report
print("\n=== marketing_summary Pipeline Health Report ===")
for k, v in report.items():
    print(f"{k}: {v}")




INFO: Loaded marketing_summary with shape (100, 50)



=== marketing_summary Pipeline Health Report ===
Missing/Broken columns detected: ['campaign_id', 'campaign_date']
Schema double check: Skipped
Error Handling check: Passed
